In [6]:
# pandas for data manipulation
import pandas as pd
pd.options.mode.chained_assignment = None
# nltk for nlp
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
# list of stopwords like articles, preposition
stop = set(stopwords.words('english'))
from string import punctuation
from collections import Counter
import re
import numpy as np
import os
trainFile=os.path.basename("news.csv")
data = pd.read_csv(trainFile, encoding = "ISO-8859-1")
data.head()


,author,description,publishedAt,source,title,url,urlToImage,category,scraping_date
0,"http://www.abc.net.au/news/julie-doyle/166932,...",The Government is separating its childcare pac...,2017-03-21T13:14:36Z,abc-news-au,"Childcare, welfare changes to be cut from omni...",http://www.abc.net.au/news/2017-03-22/governme...,http://www.abc.net.au/news/image/688364-1x1-70...,general,2017-03-21 22:37:10.566341
1,http://www.abc.net.au/news/sophie-mcneill/4516794,DFAT says an internal review into World Vision...,2017-03-21T07:15:14Z,abc-news-au,No evidence World Vision funds diverted to Ham...,http://www.abc.net.au/news/2017-03-21/no-evide...,http://www.abc.net.au/news/image/8179970-1x1-7...,general,2017-03-21 22:37:10.566341
2,http://www.abc.net.au/news/eleanor-hall/167126...,Up to 50 teenagers who were sent a video of a ...,2017-03-21T13:08:58Z,abc-news-au,Teens could face sex offenders register after ...,http://www.abc.net.au/news/2017-03-21/teens-co...,http://www.abc.net.au/news/image/7318898-1x1-7...,general,2017-03-21 22:37:10.566341
3,http://www.abc.net.au/news/lucy-marks/6524552,The Northern Territory Parliament passes a law...,2017-03-21T12:57:48Z,abc-news-au,NT passes historic abortion reform after emoti...,http://www.abc.net.au/news/2017-03-21/nt-abort...,http://www.abc.net.au/news/image/7448668-1x1-7...,general,2017-03-21 22:37:10.566341
4,http://www.abc.net.au/news/steven-schubert/274...,A judge awards compensation to four teenagers ...,2017-03-21T10:47:49Z,abc-news-au,"Don Dale teens forced into spit hoods, shackle...",http://www.abc.net.au/news/2017-03-21/don-dale...,http://www.abc.net.au/news/image/8373930-1x1-7...,general,2017-03-21 22:37:10.566341


In [7]:
data.category.value_counts().plot(kind='bar', grid=True, figsize=(16, 9))

In [8]:
print('data shape:', data.shape)
data.category.value_counts().plot(kind='bar', grid=True, figsize=(16, 9))
data = data.drop_duplicates('description')
data = data[~data['description'].isnull()]
print('data shape:', data.shape)
data['len'] = data['description'].map(len)
print(data)
data = data[data.len > 140]
data.reset_index(inplace=True)
data.drop('index', inplace=True, axis=1)



data shape: (493, 9)
data shape: (486, 9)
                                                author  \
0    http://www.abc.net.au/news/julie-doyle/166932,...   
1    http://www.abc.net.au/news/sophie-mcneill/4516794   
2    http://www.abc.net.au/news/eleanor-hall/167126...   
3        http://www.abc.net.au/news/lucy-marks/6524552   
4    http://www.abc.net.au/news/steven-schubert/274...   
5         http://www.abc.net.au/news/dan-oakes/5618988   
6                                       Sean Gallagher   
7                                            Beth Mole   
8                                          Kyle Orland   
9                                  Valentina Palladino   
10                                 Valentina Palladino   
11                                         Eric Berger   
12                                 Valentina Palladino   
13                                   Andrew Cunningham   
14                                   Andrew Cunningham   
15                            

In [9]:
def tokenizer(text):
    try:
        tokens_ = [word_tokenize(sent) for sent in sent_tokenize(text)]
        
        tokens = []
        for token_by_sent in tokens_:
            tokens += token_by_sent

        tokens = list(filter(lambda t: t.lower() not in stop, tokens))
        tokens = list(filter(lambda t: t not in punctuation, tokens))
        tokens = list(filter(lambda t: t not in [u"'s", u"n't", u"...", u"''", u'``', 
                                            u'\u2014', u'\u2026', u'\u2013'], tokens))
        filtered_tokens = []
        for token in tokens:
            if re.search('[a-zA-Z]', token):
                filtered_tokens.append(token)

        filtered_tokens = list(map(lambda token: token.lower(), filtered_tokens))

        return filtered_tokens
    except Error as e:
        print(e)
data['tokens'] = data['description'].map(tokenizer)
for descripition, tokens in zip(data['description'].head(5), data['tokens'].head(5)):
    print('description:', descripition)
    print('tokens:', tokens)
    print() 

def keywords(category):
    tokens = data[data['category'] == category]['tokens']
    alltokens = []
    for token_list in tokens:
        alltokens += token_list
    counter = Counter(alltokens)
    return counter.most_common(10)
for category in set(data['category']):
    print('category :', category)
    print('top 10 keywords:', keywords(category))
    print('---')





description: The Government is separating its childcare package and some cuts to welfare payments from its $4 billion omnibus savings bill, the ABC understands.
tokens: ['government', 'separating', 'childcare', 'package', 'cuts', 'welfare', 'payments', 'billion', 'omnibus', 'savings', 'bill', 'abc', 'understands']

description: DFAT says an internal review into World Vision funding in Gaza has uncovered nothing to suggest any diversion of government aid funding to Hamas.
tokens: ['dfat', 'says', 'internal', 'review', 'world', 'vision', 'funding', 'gaza', 'uncovered', 'nothing', 'suggest', 'diversion', 'government', 'aid', 'funding', 'hamas']

description: Up to 50 teenagers who were sent a video of a 15-year-old boy allegedly raping a girl could be placed on a sex offenders register if convicted, a child cyber safety expert says.
tokens: ['teenagers', 'sent', 'video', '15-year-old', 'boy', 'allegedly', 'raping', 'girl', 'could', 'placed', 'sex', 'offenders', 'register', 'convicted', 'c

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

# min_df is minimum number of documents that contain a term t
# max_features is maximum number of unique tokens (across documents) that we'd consider
# TfidfVectorizer preprocesses the descriptions using the tokenizer we defined above

vectorizer = TfidfVectorizer(min_df=10, max_features=10000, tokenizer=tokenizer, ngram_range=(1, 2))
vz = vectorizer.fit_transform(list(data['description']))
print('data shape:', vz.shape)
tfidf = dict(zip(vectorizer.get_feature_names(), vectorizer.idf_))
tfidf = pd.DataFrame(columns=['tfidf']).from_dict(dict(tfidf), orient='index')
tfidf.columns = ['tfidf']
tfidf.tfidf.hist(bins=50, figsize=(15,7))


data shape: (157, 21)


In [11]:
tfidf.sort_values(by=['tfidf'], ascending=True).head(30)


,tfidf
president,2.927101
trump,2.971553
new,2.971553
tuesday,3.018073
donald,3.172223
said,3.229382
would,3.290006
u.s.,3.354545
donald trump,3.354545
former,3.497646


In [12]:
tfidf.sort_values(by=['tfidf'], ascending=False).head(30)

,tfidf
could,3.664700
supreme,3.664700
made,3.664700
monday,3.664700
time,3.664700
one,3.664700
countries,3.664700
supreme court,3.664700
says,3.664700
president donald,3.577688


In [13]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=10, random_state=0)
svd_tfidf = svd.fit_transform(vz)
svd_tfidf.shape
from sklearn.manifold import TSNE

tsne_model = TSNE(n_components=2, verbose=1, random_state=0)
tsne_tfidf = tsne_model.fit_transform(svd_tfidf)
tsne_tfidf.shape
tsne_tfidf

[t-SNE] Computing pairwise distances...
[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Computed conditional probabilities for sample 157 / 157
[t-SNE] Mean sigma: 0.000000
[t-SNE] KL divergence after 100 iterations with early exaggeration: 1.633442
[t-SNE] Error after 350 iterations: 1.633442


array([[  7.83037936e+01,  -4.73268343e+01],
       [ -4.81341897e+01,  -2.17446994e+01],
       [ -5.92319159e+00,  -5.05465776e+01],
       [  1.72439516e+01,   6.67398081e+01],
       [ -2.29696935e+00,   6.14824730e+01],
       [ -5.49797788e+01,  -3.79867567e+01],
       [  5.25943928e+00,  -4.35933470e+01],
       [  2.50749322e+01,  -8.70967882e+01],
       [  5.03373389e+01,  -5.81551862e+01],
       [  2.01759789e+01,   1.53846815e+01],
       [ -4.61644046e+01,   1.46409804e+01],
       [ -3.09628966e+00,  -6.07275480e+01],
       [ -8.35551959e+01,  -1.65963912e+02],
       [ -1.82024424e+01,  -9.95085736e+00],
       [  1.08836256e+02,   1.88533842e+02],
       [  4.42353724e+01,  -5.23299525e+01],
       [ -7.29290932e+01,  -2.43166642e+01],
       [  3.96577468e+01,  -7.63884901e+01],
       [  4.85628254e+01,   1.78357131e+01],
       [ -1.39450217e+01,   2.12695484e+01],
       [  9.73988739e+00,  -7.82407036e+01],
       [  7.31348006e+01,   7.09245294e+01],
       [ -

In [14]:
import bokeh.plotting as bp
from bokeh.models import HoverTool, BoxSelectTool
from bokeh.plotting import figure, show, output_notebook
output_notebook()
plot_tfidf = bp.figure(plot_width=700, plot_height=600, title="tf-idf clustering of the news",
    tools="pan,wheel_zoom,box_zoom,reset,hover,previewsave",
    x_axis_type=None, y_axis_type=None, min_border=1)
tfidf_df = pd.DataFrame(tsne_tfidf, columns=['x', 'y'])
tfidf_df['description'] = data['description']
tfidf_df['category'] = data['category']
plot_tfidf.scatter(x='x', y='y', source=tfidf_df)
hover = plot_tfidf.select(dict(type=HoverTool))
hover.tooltips={"description": "@description", "category":"@category"}
show(plot_tfidf)

Loading BokehJS ...

In [15]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

from sklearn.cluster import MiniBatchKMeans

num_clusters = 30
kmeans_model = MiniBatchKMeans(n_clusters=num_clusters, init='k-means++', n_init=1, 
                         init_size=1000, batch_size=1000, verbose=False, max_iter=1000)
kmeans = kmeans_model.fit(vz)
kmeans_clusters = kmeans.predict(vz)
kmeans_distances = kmeans.transform(vz)
for (i, desc),category in zip(enumerate(data.description),data['category']):
    if(i < 5):
        print("Cluster " + str(kmeans_clusters[i]) + ": " + desc + 
              "(distance: " + str(kmeans_distances[i][kmeans_clusters[i]]) + ")")
        print('category: ',category)
        print('---')




Cluster 3: The Government is separating its childcare package and some cuts to welfare payments from its $4 billion omnibus savings bill, the ABC understands.(distance: 0.0)
category:  general
---
Cluster 9: DFAT says an internal review into World Vision funding in Gaza has uncovered nothing to suggest any diversion of government aid funding to Hamas.(distance: 0.245511208428)
category:  general
---
Cluster 8: Up to 50 teenagers who were sent a video of a 15-year-old boy allegedly raping a girl could be placed on a sex offenders register if convicted, a child cyber safety expert says.(distance: 0.634742928699)
category:  general
---
Cluster 3: A judge awards compensation to four teenagers over their mistreatment at Darwin's Don Dale Youth Detention Centre after a tear-gassing incident in 2014.(distance: 0.0)
category:  general
---
Cluster 4: NZ special forces soldiers in Afghanistan were involved in an operation in which a number of women and children were killed in 2010, a new book cl

In [16]:
sorted_centroids = kmeans.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(num_clusters):
    print("Cluster %d:" % i)
    aux = ''
    for j in sorted_centroids[i, :10]:
        aux += terms[j] + ' | '
    print(aux)
    print()


Cluster 0:
u.s. | countries | new | would | one | court | donald | donald trump | former | made | 

Cluster 1:
time | new | would | one | countries | court | donald | donald trump | former | made | 

Cluster 2:
president | donald | president donald | donald trump | trump | could | tuesday | monday | former | u.s. | 

Cluster 3:
would | one | countries | court | donald | donald trump | former | made | monday | new | 

Cluster 4:
new | would | one | countries | court | donald | donald trump | former | made | monday | 

Cluster 5:
tuesday | president | one | countries | court | donald | donald trump | former | made | monday | 

Cluster 6:
former | said | tuesday | would | new | countries | court | donald | donald trump | made | 

Cluster 7:
president | one | countries | court | donald | donald trump | former | made | monday | new | 

Cluster 8:
could | says | one | countries | court | donald | donald trump | former | made | monday | 

Cluster 9:
says | one | court | would | countries | do

In [17]:
tsne_kmeans = tsne_model.fit_transform(kmeans_distances)
sorted_centroids = kmeans.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(num_clusters):
    print("Cluster %d:" % i)
    aux = ''
    for j in sorted_centroids[i, :10]:
        aux += terms[j] + ' | '
    print(aux)
    print()

[t-SNE] Computing pairwise distances...
[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Computed conditional probabilities for sample 157 / 157
[t-SNE] Mean sigma: 0.000000
[t-SNE] KL divergence after 100 iterations with early exaggeration: 1.482223
[t-SNE] Error after 325 iterations: 1.482223
Cluster 0:
u.s. | countries | new | would | one | court | donald | donald trump | former | made | 

Cluster 1:
time | new | would | one | countries | court | donald | donald trump | former | made | 

Cluster 2:
president | donald | president donald | donald trump | trump | could | tuesday | monday | former | u.s. | 

Cluster 3:
would | one | countries | court | donald | donald trump | former | made | monday | new | 

Cluster 4:
new | would | one | countries | court | donald | donald trump | former | made | monday | 

Cluster 5:
tuesday | president | one | countries | court | donald | donald trump | former | made | monday | 

Cluster 6:
former | said | tuesday | would | new | countries | court |

In [18]:

tsne_kmeans = tsne_model.fit_transform(kmeans_distances)
import numpy as np

colormap = np.array(["#6d8dca", "#69de53", "#723bca", "#c3e14c", "#c84dc9", "#68af4e", "#6e6cd5",
"#e3be38", "#4e2d7c", "#5fdfa8", "#d34690", "#3f6d31", "#d44427", "#7fcdd8", "#cb4053", "#5e9981",
"#803a62", "#9b9e39", "#c88cca", "#e1c37b", "#34223b", "#bdd8a3", "#6e3326", "#cfbdce", "#d07d3c",
"#52697d", "#7d6d33", "#d27c88", "#36422b", "#b68f79"])

plot_kmeans = bp.figure(plot_width=700, plot_height=600, title="KMeans clustering of the news",
    tools="pan,wheel_zoom,box_zoom,reset,hover,previewsave",
    x_axis_type=None, y_axis_type=None, min_border=1)
kmeans_df = pd.DataFrame(tsne_kmeans, columns=['x', 'y'])
kmeans_df['cluster'] = kmeans_clusters
kmeans_df['description'] = data['description']
kmeans_df['category'] = data['category']
plot_kmeans.scatter(x='x', y='y', 
                    color=colormap[kmeans_clusters], 
                    source=kmeans_df)
hover = plot_kmeans.select(dict(type=HoverTool))
hover.tooltips={"description": "@description", "category": "@category", "cluster":"@cluster"}
show(plot_kmeans)

[t-SNE] Computing pairwise distances...
[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Computed conditional probabilities for sample 157 / 157
[t-SNE] Mean sigma: 0.000000
[t-SNE] KL divergence after 100 iterations with early exaggeration: 1.482223
[t-SNE] Error after 325 iterations: 1.482223


In [19]:
import lda
from sklearn.feature_extraction.text import CountVectorizer

In [20]:
import logging
logging.getLogger("lda").setLevel(logging.WARNING)

In [21]:
cvectorizer = CountVectorizer(min_df=4, max_features=10000, tokenizer=tokenizer, ngram_range=(1,2))
cvz = cvectorizer.fit_transform(data['description'])
n_topics = 20
n_iter = 2000
lda_model = lda.LDA(n_topics=n_topics, n_iter=n_iter)
X_topics = lda_model.fit_transform(cvz)
n_top_words = 8
topic_summaries = []
topic_word = lda_model.topic_word_  # get the topic words
vocab = cvectorizer.get_feature_names()
for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
    topic_summaries.append(' '.join(topic_words))
    print('Topic {}: {}'.format(i, ' '.join(topic_words)))

Topic 0: make company twitter social got new apple said
Topic 1: trump administration trump administration political whether hearing made leader
Topic 2: team top break big week signed announced year
Topic 3: monday trumps many deal cut russia mr. republicans
Topic 4: one would year bill saying government says back
Topic 5: people keep according plans brexit international britain british
Topic 6: court supreme court supreme tuesday gorsuch neil gorsuch neil court nominee
Topic 7: next including new england see home pick apple
Topic 8: countries middle devices flights passengers electronic electronic devices united
Topic 9: time today its like also shows social international
Topic 10: time free party war find often could former
Topic 11: president donald trump donald trump president donald pick flights get
Topic 12: could says us world one president plan review
Topic 13: new end made world league last told join
Topic 14: arsenal london north four premier league since days season
Topic

In [22]:
tsne_lda = tsne_model.fit_transform(X_topics)

[t-SNE] Computing pairwise distances...
[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Computed conditional probabilities for sample 157 / 157
[t-SNE] Mean sigma: 0.257588
[t-SNE] KL divergence after 100 iterations with early exaggeration: 1.640810
[t-SNE] Error after 400 iterations: 1.640810


In [23]:
doc_topic = lda_model.doc_topic_
lda_keys = []
for i, tweet in enumerate(data['description']):
    lda_keys += [doc_topic[i].argmax()]
plot_lda = bp.figure(plot_width=700, plot_height=600, title="LDA topic visualization",
    tools="pan,wheel_zoom,box_zoom,reset,hover,previewsave",
    x_axis_type=None, y_axis_type=None, min_border=1)
lda_df = pd.DataFrame(tsne_lda, columns=['x','y'])
lda_df['description'] = data['description']
lda_df['category'] = data['category']
lda_df['topic'] = lda_keys
lda_df['topic'] = lda_df['topic'].map(int)
plot_lda.scatter(source=lda_df, x='x', y='y', color=colormap[lda_keys])

hover = plot_lda.select(dict(type=HoverTool))
hover.tooltips={"description":"@description", "topic":"@topic", "category":"@category"}
show(plot_lda)

In [25]:
lda_df['len_docs'] = data['tokens'].map(len)
def prepareLDAData():
    data = {
        'vocab': vocab,
        'doc_topic_dists': lda_model.doc_topic_,
        'doc_lengths': list(lda_df['len_docs']),
        'term_frequency':cvectorizer.vocabulary_,
        'topic_term_dists': lda_model.components_
    } 
    return data
ldadata = prepareLDAData()
import pyLDAvis
pyLDAvis.enable_notebook()
prepared_data = pyLDAvis.prepare(**ldadata)
pyLDAvis.save_html(prepared_data,'./pyldadavis.html')